In [ ]:
from PPO_train_test import PPO_train, PPO_train_udr, PPO_train_adr, PPO_test

# Hopper Environment

We want to deploy our agent in the real world that is represented by the Target environment, but we have only access to a simulator (source environment) which has a sistematic error wrt real word.

Since we want to close this sim2real gap we will experiment some techniques to address this problem.

To begin with, we will try a simple UDR. Then our focus will be implementing an ADR technique inspired to the OpenAI original paper where ADR is formally introduced in the RL world.

In the following experiment a PPO, policy-based RL algorithm, will be employed.

Nota:
- In Target, l'hopper ha un torso più pesante. La massa elevata conferisce inerzia. L'inerzia agisce come un "filtro passa-basso" naturale: smorza le oscillazioni e rende il robot più stabile contro piccoli errori di controllo o rumore nelle azioni.

- In Source, l'hopper ha un torso molto più leggero. Con meno inerzia, il robot diventa "nervoso" (twitchy). Ogni piccola forza applicata dai motori si traduce in un'accelerazione angolare molto più rapida (F=m⋅a→a=F/m; se m scende, a sale). Questo rende molto più facile per il robot perdere l'equilibrio e cadere (terminando l'episodio prima dei 500 step).

I motori (attuatori) dell'Hopper hanno la stessa forza massima in entrambi gli ambienti, ma nel Source devono spingere meno massa.
Nel caso Source, il robot è "sovra-potenziato" rispetto al suo peso. Questo rende il paesaggio di ottimizzazione (la "superficie" che l'algoritmo PPO deve scalare) molto più frastagliato. Un'azione leggermente sbagliata nel Source porta a conseguenze catastrofiche (caduta) molto più velocemente che nel Target.
L'ambiente Hopper-v* standard di Gym (che corrisponde al tuo Target) è stato progettato e sintonizzato dai creatori di MuJoCo per avere proprietà fisiche che rendono la locomozione apprendibile e stabile. Modificando arbitrariamente una massa fondamentale come quella del torso (togliendo 1kg, che è una percentuale significativa del peso totale), si crea un robot "sbilanciato" o fisicamente meno adatto alla locomozione rispetto al design originale.

In [ ]:
TIMESTEPS = 1_000_000
SEED = 11

To begin with, lets train PPO on source and test on source:

In [ ]:
PPO_train(
    train_env_id='CustomHopper-source-v0',
    model_name=f'ppo_source_{SEED}',
    lr=3e-4,
    steps=TIMESTEPS,
    seed=SEED
)

In [ ]:
mean, std = PPO_test(
    test_env_id='CustomHopper-source-v0',
    model_name=f'ppo_source_{SEED}'
)

Now, lets test it on target:

In [ ]:
mean, std = PPO_test(
    test_env_id='CustomHopper-target-v0',
    model_name=f'ppo_source_{SEED}'
)

Now we train PPO on target and test it on target

In [ ]:
PPO_train(
    train_env_id='CustomHopper-target-v0',
    model_name=f'ppo_target_{SEED}',
    lr=3e-4,
    steps=TIMESTEPS,
    seed=SEED
)

In [ ]:
mean, std = PPO_test(
    test_env_id='CustomHopper-target-v0',
    model_name=f'ppo_target_{SEED}'
)

## UDR

Lets use UDR:

In [ ]:
PPO_train_udr(
    train_env_id='CustomHopper-source-v0',
    model_name=f'ppo_source_udr_30_{SEED}',
    lr=3e-4,
    lr_scheduler_type='constant',
    steps=TIMESTEPS,
    udr_range=0.3,
    net_size="medium", # [ small - medium - large ] -> [ 64 - 128 - 256 ],
    seed=SEED
)

In [ ]:
mean, std = PPO_test(
    test_env_id='CustomHopper-target-v0',
    model_name=f'ppo_source_udr_30_{SEED}'
)

## ADR

Now, lets get serious and use Automatic Domain Randomization (OpenAI style, that is testing the borders before enlarging the distribution range):

In [ ]:
PPO_train_adr(
    train_env_id='CustomHopper-source-v0',
    model_name=f'ppo_source_adr_30_medium_{SEED}',
    lr=3e-4,
    lr_scheduler_type="constant",
    steps=1_500_000,
    starting_adr_range=0.05,
    objective_adr_range=0.3,
    increase_rate=0.05,
    reward_to_check=1400, # forse troppo alto ?
    check_frequency=40_000, # forse troppo alto ?
    net_size="medium",
    seed=SEED
)

In [ ]:
# consigliata
PPO_train_adr(
    train_env_id='CustomHopper-source-v0',
    model_name=f'ppo_source_adr_30_large_{SEED}',
    lr=3e-4,
    lr_scheduler_type="constant",
    steps=TIMESTEPS, # 1.5M aiuterebbe
    starting_adr_range=0.05,
    objective_adr_range=0.3,
    increase_rate=0.05,
    reward_to_check=1200,   # <--- MODIFICA CRUCIALE: Lascialo salire prima
    check_frequency=20_000, # <--- Più reattivo
    net_size="large",       # <--- MODIFICA CRUCIALE: Più neuroni per gestire il caos
    seed=SEED
)

In [ ]:
mean, std = PPO_test(
    test_env_id='CustomHopper-target-v0',
    model_name=f'ppo_source_adr_30_medium_{SEED}'
)

## Visualization

In [ ]:
from utils.visualize_agent import visualize

model = f"ppo_source_adr_30_medium_{SEED}"

visualize(
    model_path=f"models/{model}",
    env_id="CustomHopper-target-v0"
)

# GRID SEARCH

In [ ]:
import importlib
import grid_search_utils

# Forza il ricaricamento del modulo modificato
importlib.reload(grid_search_utils)

from grid_search_utils import ADRGridSearch

# Definizione dei range: [start, end, step]
OBJ_ADR_GRID = [0.3, 0.7, 0.2]      # Test 0.3, 0.5, 0.7
REWARD_TH_GRID = [1200, 1500, 300]  # Test 1200, 1500, 1800
CHECK_FREQ_GRID = [40000, 80000, 40000] # Test 40k, 80k
INCREASE_RATE = [0.02, 0.03, 0.01]  # Test 0.02, 0.03, 0.01
ARCHITECTURES = ["medium", "large"]  # Test small, medium, large

# questi giù sono i parametri fissi per ogni trainig (fuori dalla grid)
LEARNING_RATE = 3e-4
LR_SCHEDULER_TYPE = "constant"
STARTING_ADR_RANGE = 0.05
TIMESTEPS = 1_000_000
SEED = 11

grid = ADRGridSearch(
    train_env_id="CustomHopper-source-v0",
    test_env_id="CustomHopper-target-v0",
    obj_adr_range=OBJ_ADR_GRID,
    reward_threshold_range=REWARD_TH_GRID,
    check_freq_range=CHECK_FREQ_GRID,
    seed=SEED,
    timesteps=TIMESTEPS,
    architectures=ARCHITECTURES,
    increase_rate=INCREASE_RATE,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type=LR_SCHEDULER_TYPE,
    starting_adr_range=STARTING_ADR_RANGE
)


grid.run_search()